<a href="https://colab.research.google.com/github/Subashganesan00/Guvi-Assignments/blob/main/tridotstech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask_sqlalchemy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///inventory.db'
db = SQLAlchemy(app)

# Database Models
class Product(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class Location(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class ProductMovement(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    timestamp = db.Column(db.DateTime, nullable=False)
    from_location = db.Column(db.Integer, db.ForeignKey('location.id'))
    to_location = db.Column(db.Integer, db.ForeignKey('location.id'))
    product_id = db.Column(db.Integer, db.ForeignKey('product.id'))
    quantity = db.Column(db.Integer, nullable=False)

    from_location_rel = db.relationship('Location', foreign_keys=[from_location])
    to_location_rel = db.relationship('Location', foreign_keys=[to_location])
    product = db.relationship('Product')

# Routes
@app.route('/')
def index():
    return 'Welcome to the Inventory Management System'

@app.route('/products', methods=['GET', 'POST'])
def products():
    if request.method == 'POST':
        name = request.form['name']
        product = Product(name=name)
        db.session.add(product)
        db.session.commit()

    products = Product.query.all()
    return render_template('products.html', products=products)

@app.route('/locations', methods=['GET', 'POST'])
def locations():
    if request.method == 'POST':
        name = request.form['name']
        location = Location(name=name)
        db.session.add(location)
        db.session.commit()

    locations = Location.query.all()
    return render_template('locations.html', locations=locations)

@app.route('/product_movement', methods=['GET', 'POST'])
def product_movement():
    if request.method == 'POST':
        from_location = request.form['from_location']
        to_location = request.form['to_location']
        product_id = request.form['product']
        quantity = request.form['quantity']

        movement = ProductMovement(
            from_location=from_location,
            to_location=to_location,
            product_id=product_id,
            quantity=quantity
        )
        db.session.add(movement)
        db.session.commit()

    movements = ProductMovement.query.all()
    return render_template('product_movement.html', movements=movements)

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)
